In [70]:
import requests, re, pandas as pd
import time, os, json
from bs4 import BeautifulSoup

TOTAL_RESTAURANT = 1000
base_url = 'https://www.tripadvisor.com/RestaurantSearch'
headers = {
    'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
}

REST_LIST_DIR = 'cache/rest_list'
REST_DIR = 'cache/restaurants'


if not os.path.exists(REST_LIST_DIR):
    os.makedirs(REST_LIST_DIR)

for idx in range(0, TOTAL_RESTAURANT, 30):
    if not os.path.exists(REST_LIST_DIR+'/NY_'+str(idx)+'.html'):
        parameter_dictionary = {'Action':'PAGE', 'ajax':'1', 'availSearchEnabled':'false', 'sortOrder':'popularity', 
                                'geo':'60763', 'itags':'10591', 'geobroaden':'false', 'o':'a'+str(idx)}
        response = requests.get(url, parameter_dictionary, headers=headers)
        f = open(REST_LIST_DIR+'/NY_'+str(idx)+'.html', 'w', encoding='utf-8')
        f.write(response.text)
        f.close()
        time.sleep(3)


In [71]:
rest_names = []
rest_urls = []
for idx in range(0, TOTAL_RESTAURANT, 30):
    f = open('cache/rest_list/NY_'+str(idx)+'.html', 'r', encoding='utf-8')
    html = f.read()
    soup = BeautifulSoup(html)
    for rest in soup.body.find_all('a', class_='Lwqic Cj b'):
        rest_names.append(rest.text)
        rest_urls.append("https://www.tripadvisor.com"+str(rest).split('"')[3])
    f.close()

In [72]:
if not os.path.exists(REST_DIR):
    os.makedirs(REST_DIR)
    
for url in rest_urls:
    file_name = url.split('-Reviews-')[1].split('-New_York_City')[0]+'.html'
    if not os.path.exists(REST_DIR+ '/'+ file_name):
        response = requests.get(url, parameter_dictionary, headers=headers)
        f = open(REST_DIR+ '/'+ file_name, 'w', encoding='utf-8')
        f.write(response.text)
        f.close()
        time.sleep(3)

In [73]:
for rest_name in os.listdir(REST_DIR):
    f = open(REST_DIR + '/' + rest_name, 'r', encoding='utf-8')
    html = f.read()
    soup = BeautifulSoup(html)
    f.close()
    
    

SyntaxError: invalid syntax (<ipython-input-73-7a8b89bc8058>, line 2)